In [29]:
import os
import pandas as pd
import janitor

contratos = os.path.join('..', 'data', '2023.xlsx')

contratos_df=pd.read_excel(contratos)
contratos_df= contratos_df.clean_names()


In [30]:
print(
    'El número de contratos es de',
    contratos_df.shape[0]
)

El número de contratos es de 12567


In [31]:
pd.DataFrame(contratos_df.dtypes)

,0
orden_de_gobierno,object
siglas_de_la_institucion,object
institucion,object
clave_de_la_uc,object
nombre_de_la_uc,object
responsable_de_la_uc,object
codigo_del_expediente,int64
referencia_del_expediente,object
clave_cucop,object
titulo_del_expediente,object


In [32]:
for _ in contratos_df.columns:
    print(_)

orden_de_gobierno
siglas_de_la_institucion
institucion
clave_de_la_uc
nombre_de_la_uc
responsable_de_la_uc
codigo_del_expediente
referencia_del_expediente
clave_cucop
titulo_del_expediente
plantilla_del_expediente
fundamento_legal
numero_del_procedimiento
fecha_de_fallo
fecha_de_publicacion
fecha_de_apertura
caracter_del_procedimiento
tipo_de_contratacion
tipo_de_procedimiento
forma_de_participacion
codigo_del_contrato
num_de_control_del_contrato
titulo_del_contrato
descripcion_del_contrato
fecha_de_inicio_del_contrato
fecha_de_fin_del_contrato
importe_del_contrato
moneda_del_contrato
estatus_del_contrato
convenio_modificatorio
clave_del_programa_federal
fecha_de_firma_del_contrato
contrato_marco
compra_consolidada
contrato_plurianual
clave_de_cartera_shcp
folio_en_el_rupc
rfc
proveedor_o_contratista
estratificacion_de_la_empresa
clave_del_pais_de_la_empresa
rfc_verificado_en_el_sat
credito_externo
organismo_financiero
direccion_del_anuncio


### Limpiando datos

In [33]:
from currency_converter import CurrencyConverter


for column in contratos_df.columns:
    if contratos_df[column].dtype == 'object':
        contratos_df[column] = contratos_df[column].str.upper()


for column in contratos_df.columns:
    if contratos_df[column].dtype == 'string':
        contratos_df[column] = contratos_df[column].str.upper()

c = CurrencyConverter()
currencies=['USD', 'CAD', 'EUR', 'GBP', 'JPY']

# Create a dictionary to store conversion rates
conversion_rates = {}

for m in currencies:
    conversion_rate = c.convert(1, m, 'MXN')
    print(1, m, "=", conversion_rate, 'MXN')
    conversion_rates[m] = conversion_rate

# Apply conversion rates
contratos_df["importe"] = contratos_df["importe_del_contrato"]

for currency, conversion_rate in conversion_rates.items():
    contratos_df.loc[contratos_df['moneda_del_contrato'] == currency , 'importe'] = contratos_df.loc[contratos_df['moneda_del_contrato'] == currency , "importe_del_contrato"] * conversion_rate

print('Done')


1 USD = 17.115130976369297 MXN
1 CAD = 13.018183084854396 MXN
1 EUR = 18.6863 MXN
1 GBP = 21.75912340762477 MXN
1 JPY = 0.11946998273767662 MXN
Done


### Contratos acumulados por instituciones

In [34]:
pd.options.display.float_format = '{:,}'.format

ins_df=contratos_df.groupby(['siglas_de_la_institucion', 'institucion']).agg({'importe':'sum'}).reset_index()

ins_df.sort_values(by='importe', ascending=False, inplace=True)
#institucion.set_index('Institución', inplace=True)
ins_df

,siglas_de_la_institucion,institucion,importe
156,IMSS,INSTITUTO MEXICANO DEL SEGURO SOCIAL,"128,606,012,474.8178"
0,AEFCM,AUTORIDAD EDUCATIVA FEDERAL EN LA CIUDAD DE MÉ...,"104,040,581,677.53"
123,FIDENA,FIDEICOMISO DE FORMACIÓN Y CAPACITACIÓN PARA E...,"77,259,373,114.01"
200,ISSSTE,INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE...,"32,185,592,525.299"
193,INSABI,INSTITUTO DE SALUD PARA EL BIENESTAR,"21,571,169,082.55295"
...,...,...,...
275,VER,_GOBIERNO MUNICIPAL DEL ESTADO DE VERACRUZ DE ...,"62,500.0"
243,SESNA,SECRETARÍA EJECUTIVA DEL SISTEMA NACIONAL ANTI...,"33,687.0"
77,CNTS,CENTRO NACIONAL DE LA TRANSFUSIÓN SANGUÍNEA,"20,700.0"
92,CONADIS,CONSEJO NACIONAL PARA EL DESARROLLO Y LA INCLU...,"11,945.99"


### Contratos acumulados por empresas

In [35]:
pd.options.display.float_format = '{:,}'.format

prov_df=contratos_df.groupby(['proveedor_o_contratista', 'rfc']).agg({'importe':'sum'}).reset_index()

prov_df.sort_values(by='importe', ascending=False, inplace=True)

def convert_year(two_digit_year):
    if int(two_digit_year) > 23:
        return "19" + two_digit_year
    else:
        return "20" + two_digit_year

prov_df['año_constitucion'] = prov_df['rfc'].astype(str).str[3:5].apply(lambda x: convert_year(x))

#institucion.set_index('Institución', inplace=True)
prov_df

,proveedor_o_contratista,rfc,importe,año_constitucion
2097,WYNNMEX TECHNOLOGIES S DE RL DE CV,WTE1307027J7,"77,259,004,011.0",2013
90,AM CENIT SA DE CV,ACE120723464,"27,976,369,404.31927",2012
1314,LMR GRUP SA DE CV,LGR061118PA1,"21,479,991,573.59491",2006
2016,TOPO TACTICO SA DE CV,TTA210305S83,"15,522,751,396.59",2021
2007,TEXTILERA Y PROVEEDORA ATZIN SA DE CV,TPA171026R10,"12,533,685,717.119999",2017
...,...,...,...,...
1459,MUNICIPIO DE CULIACAN,MCU9710067R9,50.968,1997
124,ASEGURADORA PATRIMONIAL VIDA SA DE CV,APV040521RP0,0.0,2004
1807,SERVICIOS ADMINISTRATIVOS Y EMPRESARIALES DEL ...,SAE120524LE3,0.0,2012
1754,SAYSE S DE RL DE CV,SAY080911R82,0.0,2008


In [59]:
import numpy as np

# Primero agrupamos por 'proveedor_o_contratista', 'rfc', y 'siglas_de_la_institucion'
grouped_df = contratos_df.groupby(['proveedor_o_contratista', 'rfc', 'siglas_de_la_institucion']).agg({'importe':'sum'}).reset_index()

# Luego hacemos pivot sobre el dataframe agrupado
pivot_df = pd.pivot_table(grouped_df, values='importe', index=['proveedor_o_contratista', 'rfc'], columns='siglas_de_la_institucion', fill_value=0)

# Finalmente reseteamos el índice
prov_df = pivot_df.reset_index()

# Agregar columna de importe total
prov_df['importe_total'] = prov_df.loc[:, prov_df.columns != 'rfc'].sum(axis=1, numeric_only=True)

prov_df.loc[:, 'año_constitucion'] = prov_df['rfc'].astype(str).str[3:5].apply(lambda x: convert_year(x))


# Reordenar las columnas para que 'importe_total' aparezca primero
column_order = ['proveedor_o_contratista', 'rfc', 'importe_total', 'año_constitucion'] + [col for col in prov_df.columns if col not in ['proveedor_o_contratista', 'rfc', 'importe_total', 'año_constitucion']]
prov_df = prov_df[column_order]

# Añadir año_constitucion

# Ordenar por importe_total
prov_df.sort_values(by='importe_total', ascending=False, inplace=True)
prov_df = prov_df.reset_index()

#prov_df.drop(columns='siglas_de_la_institucion', inplace=True)

prov_df


siglas_de_la_institucion,index,proveedor_o_contratista,rfc,importe_total,año_constitucion,AEFCM,AFAC,AGN,AGROASEMEX,AGS,...,TECNM,TFJA,TLAX,TRIBUNALES AGRARIOS,TVMETRO,UNADM,UPN,USCMM,VER,ZAC
0,2097,WYNNMEX TECHNOLOGIES S DE RL DE CV,WTE1307027J7,"77,259,004,011.0",2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90,AM CENIT SA DE CV,ACE120723464,"27,976,369,404.319267",2012,"27,910,160,440.0","39,183.0","7,362.3",0.0,0.0,...,0.0,0.0,0.0,0.0,"45,889.75","3,873.15",0.0,0.0,0.0,0.0
2,1314,LMR GRUP SA DE CV,LGR061118PA1,"21,479,991,573.594906",2006,"21,336,359,000.0","40,898.0","75,389.2",0.0,0.0,...,0.0,0.0,0.0,0.0,"29,765.1","20,033.88",0.0,"128,945.44",0.0,0.0
3,2016,TOPO TACTICO SA DE CV,TTA210305S83,"15,522,751,396.589996",2021,"15,492,162,400.0",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2007,TEXTILERA Y PROVEEDORA ATZIN SA DE CV,TPA171026R10,"12,533,685,717.119995",2017,"12,529,221,020.0",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,"5,175.76",0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100,1459,MUNICIPIO DE CULIACAN,MCU9710067R9,50.968,1997,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2101,124,ASEGURADORA PATRIMONIAL VIDA SA DE CV,APV040521RP0,0.0,2004,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2102,1807,SERVICIOS ADMINISTRATIVOS Y EMPRESARIALES DEL ...,SAE120524LE3,0.0,2012,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2103,1754,SAYSE S DE RL DE CV,SAY080911R82,0.0,2008,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
